In [13]:
import os
import pandas as pd
import numpy as np

In [14]:
def process_stock_xlsx(file_path):
    # Load ONLY the "Data Sheet"
    df = pd.read_excel(file_path, sheet_name="Data Sheet", engine="openpyxl")
    df = df.dropna(how='all').dropna(axis=1, how='all')
    df.reset_index(drop=True, inplace=True)
    # Find the row indices for section headers
    pnl_idx = df[df['COMPANY NAME'] == 'PROFIT & LOSS'].index[0]
    q_idx = df[df['COMPANY NAME'] == 'Quarters'].index[0]
    bs_idx = df[df['COMPANY NAME'] == 'BALANCE SHEET'].index[0]
    cf_idx = df[df['COMPANY NAME'] == 'CASH FLOW:'].index[0]
    # Extract sub-DataFrames (excluding the header rows themselves)
    df_pnl = df.iloc[pnl_idx+1 : q_idx].reset_index(drop=True)
    df_q = df.iloc[q_idx+1 : bs_idx].reset_index(drop=True)
    df_bs = df.iloc[bs_idx+1 : cf_idx].reset_index(drop=True)
    df_cf = df.iloc[cf_idx+1 :].reset_index(drop=True)
    # Set column headers
    for d in (df_pnl, df_q, df_bs, df_cf):
        d.columns = d[d['COMPANY NAME']=='Report Date'].loc[0].to_list()
        d.columns.values[0]= 'Variable'
        d.drop(d.index[0], inplace=True)
        d.reset_index(drop=True, inplace=True)
    # Clean up: drop fully empty columns/rows and reset index
    for d in (df_pnl, df_bs, df_cf):
        d.dropna(how='all', axis=1, inplace=True)
        d.dropna(how='all', inplace=True)
        d.reset_index(drop=True, inplace=True)
        d.columns.values[0] = "Variable"
    # Convert column names to datetime wherever possible
    def convert_columns_to_datetime(df):
        new_cols = []
        for col in df.columns:
            try:
                new_col = pd.to_datetime(col)
            except:
                new_col = col
            new_cols.append(new_col)
        df.columns = new_cols
    convert_columns_to_datetime(df_pnl)
    convert_columns_to_datetime(df_bs)
    convert_columns_to_datetime(df_cf)
    # Normalize the 'Variable' column (strip whitespace and lowercase)
    for d in (df_pnl, df_bs, df_cf):
        d["Variable"] = d["Variable"].astype(str).str.strip().str.lower()
    # Identify all datetime columns (the last 9 fiscal years)
    years = [col for col in df_pnl.columns if isinstance(col, pd.Timestamp)]
    # Helper to fetch a row’s values across all 9 years
    def get_row_values(df, var_name):
        var_name_clean = var_name.lower().strip()
        row = df[df["Variable"] == var_name_clean]
        if row.empty:
            raise ValueError(f"Variable '{var_name}' not found")
        return row.iloc[0][years].astype(float).values
    # Extract series for the last 9 years (sales, net profit, price)
    sales = get_row_values(df_pnl, "sales")
    net_profit = get_row_values(df_pnl, "net profit")
    stock_price = get_row_values(df_cf, "price:")
    # Compute EBITDA for last 9 years = PBT + Depreciation + Interest
    ebitda = (
        get_row_values(df_pnl, "profit before tax")
        + get_row_values(df_pnl, "depreciation")
        + get_row_values(df_pnl, "interest")
    )
    # Dividend Amount (crores) – last 9 years
    div_amt = get_row_values(df_pnl, "dividend amount")
    # Number of Equity Shares (absolute count) for last 9 years
    num_shares = df_bs[df_bs["Variable"] == "no. of equity shares"][years].astype(float).values.flatten()
    # Market Cap = price * num_shares
    market_cap = stock_price * num_shares
    # Total Equity (crores) = Equity Share Capital + Reserves
    equity_cap = get_row_values(df_bs, "equity share capital")
    reserves = get_row_values(df_bs, "reserves")
    total_equity = equity_cap + reserves  # in crores
    # Book Value per Share (Rs) = (Total Equity crores * 1e7) / number of shares
    bvps = (total_equity * 1e7) / num_shares
    # P/E Ratio = Price / (Net Profit per share)
    eps = (net_profit * 1e7) / num_shares
    pe_ratio = stock_price / eps
    # P/B Ratio = Price / BVPS
    pb_ratio = stock_price / bvps
    # ROE (%) = (Net Profit crores / Total Equity crores) * 100
    roe = (net_profit / total_equity) * 100
    # ROCE (%) = [EBITDA crores / (Total Equity + Borrowings) crores] * 100
    borrowings = get_row_values(df_bs, "borrowings")
    roce = (ebitda / (total_equity + borrowings)) * 100
    # Cash Flow (Net Cash Flow) last 9 years
    cash_flow = get_row_values(df_cf, "net cash flow")
    # Net Assets = Total Equity (crores) for last 9 years
    net_assets = total_equity
    # Dividend Yield (%) = [Dividend per share / Price] * 100
    dps = (div_amt * 1e7) / num_shares
    div_yield = (dps / stock_price) * 100
    # Compute Sales % YoY increase for 8 intervals (2017 vs 2016, ..., 2025 vs 2024)
    sales_pct_yoy = [
        (sales[i] - sales[i - 1]) / sales[i - 1] * 100 if sales[i - 1] != 0 else np.nan
        for i in range(1, len(sales))
    ]
    # Compute Net Profit % YoY increase similarly
    net_profit_pct_yoy = [
        (net_profit[i] - net_profit[i - 1]) / net_profit[i - 1] * 100 if net_profit[i - 1] != 0 else np.nan
        for i in range(1, len(net_profit))
    ]

    # Debt to Equity Ratio
    de_ratio = borrowings / total_equity
    # Prepare a DataFrame or print results
    results = pd.DataFrame({
        "Year": [col.year for col in years],
        "Sales": sales,
        "Sales YoY %": [np.nan] + [round(pct, 2) for pct in sales_pct_yoy],
        "Net Profit": net_profit,
        "Net Profit YoY %": [np.nan] + [round(pct, 2) for pct in net_profit_pct_yoy],
        "Stock Price": stock_price,
        "Market Cap": market_cap,
        "EBITDA": ebitda,
        "Dividend Amt": div_amt,
        "Book Value/Share": bvps,
        "P/E": pe_ratio,
        "eps": eps,
        "P/B": pb_ratio,
        "Debt/Equity": de_ratio,
        "ROE (%)": roe,
        "ROCE (%)": roce,
        "Net Assets": net_assets,
        "Cash Flow": cash_flow,
        "Div Yield (%)": div_yield
    })
    return results

# Directory containing your xlsx files
portfolio_dir = './portfolio_files_screener_in'
results_dict = {}

for fname in os.listdir(portfolio_dir):
    if fname.endswith('.xlsx'):
        fpath = os.path.join(portfolio_dir, fname)
        try:
            results_dict[fname] = process_stock_xlsx(fpath)
            print(f"Processed: {fname}")
        except Exception as e:
            print(f"Error processing {fname}: {e}")


# Now results_dict contains a DataFrame for each stock file

Processed: Ashoka Buildcon.xlsx
Processed: Shriram Finance.xlsx
Processed: Eicher Motors.xlsx
Processed: Coal India.xlsx
Processed: Fortis Health.xlsx
Processed: Apollo Hospitals.xlsx
Processed: Tata Steel.xlsx
Error processing PNGS Gargi FJ.xlsx: "[Timestamp('2021-03-31 00:00:00')] not in index"
Processed: TVS Motor Co.xlsx
Processed: Volt.Transform.xlsx
Processed: Bluestone Jewel.xlsx
Processed: HDFC Bank.xlsx
Processed: Reliance Industr.xlsx
Processed: NTPC.xlsx
Processed: Natco Pharma.xlsx
Processed: Eternal Ltd.xlsx
Processed: Banco Products.xlsx
Processed: Bajaj Auto.xlsx
Processed: Natl. Aluminium.xlsx
Processed: Ather Energy.xlsx
Processed: J K Cements.xlsx
Processed: Shakti Pumps.xlsx
Processed: ICICI Bank.xlsx
Processed: Waaree Energies.xlsx
Processed: Coforge.xlsx
Processed: Interglobe Aviat.xlsx
Processed: Ambuja Cements.xlsx
Processed: JSW Energy.xlsx
Processed: Adani Power.xlsx
Processed: Narayana Hrudaya.xlsx
Processed: Persistent Sys.xlsx
Processed: Torrent Pharma.xlsx


In [15]:
results_dict['Adani Power.xlsx']

,Year,Sales,Sales YoY %,Net Profit,Net Profit YoY %,Stock Price,Market Cap,EBITDA,Dividend Amt,Book Value/Share,P/E,eps,P/B,Debt/Equity,ROE (%),ROCE (%),Net Assets,Cash Flow,Div Yield (%)
0,2016,25377.33,NaN,550.80,NaN,6.89,2.297084e+10,8600.76,NaN,22.399510,4.170450,1.652100,0.307596,7.060834,7.375607,14.287646,7467.86,-147.85,NaN
1,2017,22615.51,-10.88,-6174.10,-1220.93,7.98,3.077837e+10,2313.92,NaN,7.777048,-0.498508,-16.007772,1.026096,17.497356,-205.833522,4.170433,2999.56,-25.77,NaN
2,2018,20304.28,-10.22,-2102.95,-65.94,4.74,1.828189e+10,6190.03,NaN,2.305792,-0.869345,-5.452381,2.055693,59.409679,-236.464529,11.521878,889.33,-19.39,NaN
3,2019,23884.18,17.63,-984.40,-53.19,9.64,3.718089e+10,7431.28,NaN,19.995883,-3.777011,-2.552283,0.482099,6.091537,-12.764043,13.587511,7712.29,-37.08,NaN
4,2020,26467.72,10.82,-2274.77,131.08,5.55,2.140601e+10,6056.40,NaN,16.802029,-0.941019,-5.897864,0.330317,8.517749,-35.102092,9.819194,6480.44,916.56,NaN
5,2021,26221.48,-0.93,1269.98,-155.83,17.01,6.560653e+10,10596.72,NaN,33.997401,5.165950,3.292715,0.500332,3.997019,9.685196,16.172307,13112.59,-828.06,NaN
6,2022,27711.18,5.68,4911.58,286.74,37.02,1.427839e+11,13789.45,NaN,47.853726,2.907087,12.734399,0.773607,2.662690,26.611092,20.398030,18456.89,669.33,NaN
7,2023,38773.30,39.92,10726.64,118.39,38.32,1.477979e+11,14311.88,NaN,77.459510,1.377858,27.811278,0.494710,1.417526,35.904278,19.815635,29875.66,-433.14,NaN
8,2024,50351.25,29.86,20828.79,94.18,106.76,4.117668e+11,28110.93,NaN,111.224162,1.976912,54.003422,0.959863,0.812665,48.553678,36.150613,42898.48,787.02,NaN
9,2025,56203.09,11.62,12938.77,-37.88,101.86,3.928678e+11,24008.18,NaN,146.092772,3.036361,33.546733,0.697228,0.700921,22.962623,25.049759,56347.09,-816.39,NaN


In [16]:
def get_latest_valid_row(df, key_metrics=None):
    if key_metrics is None:
        key_metrics = ['P/E', 'P/B', 'ROCE (%)', 'Debt/Equity', 'Cash Flow', 'ROE (%)', 'Sales YoY %', 'Net Profit YoY %']
    # Sort by year descending
    df_sorted = df.sort_values('Year', ascending=False)
    for _, row in df_sorted.iterrows():
        if all(pd.notnull(row[m]) for m in key_metrics):
            return row
    # If none found, return the most recent row anyway
    return df_sorted.iloc[0]

def buy_score(row):
    score = 0
    score += max(row['Sales YoY %'], 0) * 0.2
    score += max(row['Net Profit YoY %'], 0) * 0.2
    score += (1 / row['P/E'] if row['P/E'] > 0 else 0) * 0.15
    score += (1 / row['P/B'] if row['P/B'] > 0 else 0) * 0.1
    score += row['ROE (%)'] * 0.15
    score += row['ROCE (%)'] * 0.1
    # score += row['Div Yield (%)'] * 0.1
    return score

def enhanced_buy_score(row, sector_pe=None, sector_pb=None):
    score = 0
    # Growth (multi-year average preferred)
    score += max(row['Sales YoY %'], 0) * 0.15
    score += max(row['Net Profit YoY %'], 0) * 0.15
    # Value (relative to sector if possible)
    score += (1 / row['P/E'] if row['P/E'] > 0 else 0) * 0.10
    score += (1 / row['P/B'] if row['P/B'] > 0 else 0) * 0.10
    # Profitability
    score += row['ROE (%)'] * 0.15
    score += row['ROCE (%)'] * 0.10
    # Cash flow
    score += (row['Cash Flow'] / abs(row['Net Profit'])) * 0.10 if row['Net Profit'] != 0 else 0
    # Leverage
    score -= (row['Debt/Equity'] * 0.10) if 'Debt/Equity' in row else 0
    # Dividend

    score += (row['Div Yield (%)'] * 0.05 if not np.isnan(row['Div Yield (%)']) else 0)

    return score


scores = {}
for stock, df in results_dict.items():
    try:
        row = get_latest_valid_row(df)
        scores[stock] = buy_score(row)
        # scores[stock] = enhanced_buy_score(row, sector_pe=None, sector_pb=None)
        # if stock == 'Eternal Ltd.xlsx':
        #     import pdb; pdb.set_trace()
    except Exception as e:
        scores[stock] = None
        print(f"Could not score {stock}: {e}")

# Sort stocks by score
sorted_scores = sorted(scores.items(), key=lambda x: x[1] if x[1] is not None else -float('inf'), reverse=True)
for x in sorted_scores:
    print(x)

('Bluestone Jewel.xlsx', np.float64(nan))
('Bharti Airtel.xlsx', np.float64(80.61955392752107))
('Shakti Pumps.xlsx', np.float64(64.28924513344302))
('Ashoka Buildcon.xlsx', np.float64(59.68016550907853))
('Bondada Engineer.xlsx', np.float64(55.96435819185077))
('Natl. Aluminium.xlsx', np.float64(47.410174575761516))
('ITC.xlsx', np.float64(29.81592016345993))
('Netweb Technol.xlsx', np.float64(28.2972262788643))
('Eternal Ltd.xlsx', np.float64(24.255382349872527))
('Waaree Energies.xlsx', np.float64(21.42783245721402))
('Banco Products.xlsx', np.float64(20.117997722826622))
('Cholaman.Inv.&Fn.xlsx', np.float64(18.968628058092676))
('Interglobe Aviat.xlsx', np.float64(17.950090819162686))
('Eicher_Motors.xlsx', np.float64(16.878045448503002))
('Natco Pharma.xlsx', np.float64(16.340748524814366))
('ICICI Bank.xlsx', np.float64(15.707308600716004))
('Narayana Hrudaya.xlsx', np.float64(15.095816565767073))
('TVS Motor Co.xlsx', np.float64(15.004205830197707))
('Shriram Finance.xlsx', np.f